# dependencies
upload the Reacher-v4 agent to /content/ppo_Reacher-v4.zip

In [1]:
#!git clone http://github.com/HumanCompatibleAI/imitation
#!pip install -e "/content/imitation[dev]"

In [2]:
!pip install gymnasium[mujoco]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 kB 4.8 MB/s eta 0:00:00


In [3]:
!pip install imitation
!pip install seals
!pip install shimmy
!pip install git+https://github.com/Stable-Baselines-Team/stable-baselines3-contrib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.4/216.4 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.7/181.7 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.9/107.9 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 14.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/

# environment and expert

In [4]:
import gymnasium as gym
import imitation
import seals

In [5]:
import numpy as np
from imitation.policies.serialize import load_policy
from imitation.util.util import make_vec_env
from imitation.data.wrappers import RolloutInfoWrapper
from imitation.algorithms.adversarial.gail import GAIL
from imitation.rewards.reward_nets import BasicRewardNet
from imitation.util.networks import RunningNorm
from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.evaluation import evaluate_policy
import sb3_contrib
from sb3_contrib import TRPO
from imitation.data import rollout
SEED = 42

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
def make_env_expert_rollouts(n_trajs):
    # Create the environment, load expert policy, and sample trajectories
    # from the expert policy and environment.
    global SEED
    env = make_vec_env(
        "Reacher-v4", # seals:seals/CartPole-v0
        rng=np.random.default_rng(SEED),
        n_envs=1,
        post_wrappers=[
            lambda env, _: RolloutInfoWrapper(env)
        ],  # needed for computing rollouts later
        parallel=True,
    )
    expert = load_policy(
        "ppo",
        path = "/content/ppo_Reacher-v4.zip", #-self-trained Reacher expert
        venv=env,
    )
    rollouts = rollout.rollout(
        expert,
        env,
        rollout.make_sample_until(min_timesteps=50, min_episodes = n_trajs),
        rng=np.random.default_rng(SEED), #The random state to use for sampling trajectories
    )

    return env, expert, rollouts


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
import torch
def make_PPO_learner_and_trainer(env, expert, rollouts):
    global SEED
# 可以调policy network的架构
# Custom actor (pi) and value function (vf) networks
# of two layers of size 32 each with Relu activation function
#policy_kwargs = dict(activation_fn=th.nn.ReLU,
#                     net_arch=[dict(pi=[32, 32], vf=[32, 32])])
    learner_PPO = PPO(
        env=env,
        policy=MlpPolicy,
        batch_size=128,
        #clip_range=0.3,
        #ent_coef= 0.0007566389899529574,
        learning_rate=1e-4, #1.943992487657563e-5
        gae_lambda=0.8,
        max_grad_norm=0.9,
        gamma=0.995,
        n_epochs=10,
        seed=SEED,
    )
    load_from_saved_policy = torch.load("BC_MlpPolicy_Reacher_1.pth.tar")
    learner_PPO.policy.load_state_dict(load_from_saved_policy.state_dict(), strict=False)

    reward_net = BasicRewardNet(
        observation_space=env.observation_space,
        action_space=env.action_space,
        normalize_input_layer=RunningNorm,
    )
    gail_trainer = GAIL(
        demonstrations = rollouts, # expert trajectories
        demo_batch_size = 50,
        gen_replay_buffer_capacity = 16384,
        n_disc_updates_per_round = 8, #
        venv = env,
        gen_algo = learner_PPO,
        reward_net=reward_net,
        allow_variable_horizon=True,
        # gen_train_timesteps = 1000
        init_tensorboard=True,
        init_tensorboard_graph=True,

    )
    return learner_PPO, gail_trainer

# GAIL-PPO 1 trajectory - 1 traj BC

In [8]:
env_1, expert_1, rollouts_1 = make_env_expert_rollouts(1)

In [9]:
learner_PPO_1, gail_trainer_PPO_1 = make_PPO_learner_and_trainer(env_1, expert_1, rollouts_1)

Running with `allow_variable_horizon` set to True. Some algorithms are biased towards shorter or longer episodes, which may significantly confound results. Additionally, even unbiased algorithms can exploit the information leak from the termination condition, producing spuriously high performance. See https://imitation.readthedocs.io/en/latest/getting-started/variable-horizon.html for more information.


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
expert_reward, _ = evaluate_policy(
    expert_1, env_1, 20, return_episode_rewards=True
)
print(
    "expert rewards after training:",
    np.mean(expert_reward),
    "+/-",
    np.std(expert_reward),
)

expert rewards after training: -5.0036284 +/- 1.6795412761026864


In [11]:
env_1.seed(SEED)
learner_rewards_before_training, _ = evaluate_policy(
    learner_PPO_1, env_1, 20, return_episode_rewards=True
)

In [12]:
gail_trainer_PPO_1.train(100000)
# An upper bound on the number of transitions to sample from the environment during training.

round:   0%|          | 0/48 [00:00<?, ?it/s]

------------------------------------------
| raw/                        |          |
|    gen/rollout/ep_len_mean  | 50       |
|    gen/rollout/ep_rew_mean  | -37.7    |
|    gen/time/fps             | 356      |
|    gen/time/iterations      | 1        |
|    gen/time/time_elapsed    | 5        |
|    gen/time/total_timesteps | 2048     |
------------------------------------------
--------------------------------------------------
| raw/                                |          |
|    disc/disc_acc                    | 0.6      |
|    disc/disc_acc_expert             | 1        |
|    disc/disc_acc_gen                | 0.2      |
|    disc/disc_entropy                | 0.692    |
|    disc/disc_loss                   | 0.69     |
|    disc/disc_proportion_expert_pred | 0.9      |
|    disc/disc_proportion_expert_true | 0.5      |
|    disc/global_step                 | 1        |
|    disc/n_expert                    | 50       |
|    disc/n_generated                 | 50       |
-

round:   2%|▏         | 1/48 [00:07<06:10,  7.88s/it]

----------------------------------------------------
| raw/                               |             |
|    gen/rollout/ep_len_mean         | 50          |
|    gen/rollout/ep_rew_mean         | -38.6       |
|    gen/rollout/ep_rew_wrapped_mean | 33          |
|    gen/time/fps                    | 369         |
|    gen/time/iterations             | 1           |
|    gen/time/time_elapsed           | 5           |
|    gen/time/total_timesteps        | 4096        |
|    gen/train/approx_kl             | 0.008735466 |
|    gen/train/clip_fraction         | 0.0977      |
|    gen/train/clip_range            | 0.2         |
|    gen/train/entropy_loss          | -1.7        |
|    gen/train/explained_variance    | 0.251       |
|    gen/train/learning_rate         | 0.0001      |
|    gen/train/loss                  | 3.31        |
|    gen/train/n_updates             | 10          |
|    gen/train/policy_gradient_loss  | -0.018      |
|    gen/train/std                   | 0.564  

round:   4%|▍         | 2/48 [00:14<05:19,  6.95s/it]

...

round:  96%|█████████▌| 46/48 [04:59<00:12,  6.46s/it]

----------------------------------------------------
| raw/                               |             |
|    gen/rollout/ep_len_mean         | 50          |
|    gen/rollout/ep_rew_mean         | -33.6       |
|    gen/rollout/ep_rew_wrapped_mean | 3.06        |
|    gen/time/fps                    | 383         |
|    gen/time/iterations             | 1           |
|    gen/time/time_elapsed           | 5           |
|    gen/time/total_timesteps        | 96256       |
|    gen/train/approx_kl             | 0.013231348 |
|    gen/train/clip_fraction         | 0.149       |
|    gen/train/clip_range            | 0.2         |
|    gen/train/entropy_loss          | -1.08       |
|    gen/train/explained_variance    | 0.99        |
|    gen/train/learning_rate         | 0.0001      |
|    gen/train/loss                  | 0.0815      |
|    gen/train/n_updates             | 460         |
|    gen/train/policy_gradient_loss  | -0.0161     |
|    gen/train/std                   | 0.414  

round:  98%|█████████▊| 47/48 [05:05<00:06,  6.37s/it]

----------------------------------------------------
| raw/                               |             |
|    gen/rollout/ep_len_mean         | 50          |
|    gen/rollout/ep_rew_mean         | -33.8       |
|    gen/rollout/ep_rew_wrapped_mean | 2.55        |
|    gen/time/fps                    | 352         |
|    gen/time/iterations             | 1           |
|    gen/time/time_elapsed           | 5           |
|    gen/time/total_timesteps        | 98304       |
|    gen/train/approx_kl             | 0.018121123 |
|    gen/train/clip_fraction         | 0.205       |
|    gen/train/clip_range            | 0.2         |
|    gen/train/entropy_loss          | -1.07       |
|    gen/train/explained_variance    | 0.989       |
|    gen/train/learning_rate         | 0.0001      |
|    gen/train/loss                  | 0.108       |
|    gen/train/n_updates             | 470         |
|    gen/train/policy_gradient_loss  | -0.0207     |
|    gen/train/std                   | 0.412  

round: 100%|██████████| 48/48 [05:11<00:00,  6.50s/it]


In [13]:
env_1.seed(SEED)
learner_rewards_after_training, _ = evaluate_policy(
    learner_PPO_1, env_1, 20, return_episode_rewards=True
)

In [14]:
print(
    "Rewards before training:",
    np.mean(learner_rewards_before_training),
    "+/-",
    np.std(learner_rewards_before_training),
)
print(
    "Rewards after training:",
    np.mean(learner_rewards_after_training),
    "+/-",
    np.std(learner_rewards_after_training),
)

Rewards before training: -5.9576569500000005 +/- 2.6330277433453007
Rewards after training: -11.31138005 +/- 4.706499342660312


# GAIL-PPO 4 trajectory - 1 traj BC

In [60]:
env_4, expert_4, rollouts_4 = make_env_expert_rollouts(4)
learner_PPO_4, gail_trainer_PPO_4 = make_PPO_learner_and_trainer(env_4, expert_4, rollouts_4)

Running with `allow_variable_horizon` set to True. Some algorithms are biased towards shorter or longer episodes, which may significantly confound results. Additionally, even unbiased algorithms can exploit the information leak from the termination condition, producing spuriously high performance. See https://imitation.readthedocs.io/en/latest/getting-started/variable-horizon.html for more information.


In [61]:
expert_reward, _ = evaluate_policy(
    expert_4, env_4, 20, return_episode_rewards=True
)
print(
    "expert rewards after training:",
    np.mean(expert_reward),
    "+/-",
    np.std(expert_reward),
)

expert rewards after training: -5.0036284 +/- 1.6795412761026864


In [62]:
env_4.seed(SEED)
learner_rewards_before_training, _ = evaluate_policy(
    learner_PPO_4, env_4, 20, return_episode_rewards=True
)

In [63]:
gail_trainer_PPO_4.train(100000) # 1000000

round:   0%|          | 0/48 [00:00<?, ?it/s]

------------------------------------------
| raw/                        |          |
|    gen/rollout/ep_len_mean  | 50       |
|    gen/rollout/ep_rew_mean  | -37.7    |
|    gen/time/fps             | 375      |
|    gen/time/iterations      | 1        |
|    gen/time/time_elapsed    | 5        |
|    gen/time/total_timesteps | 2048     |
------------------------------------------
--------------------------------------------------
| raw/                                |          |
|    disc/disc_acc                    | 0.58     |
|    disc/disc_acc_expert             | 1        |
|    disc/disc_acc_gen                | 0.16     |
|    disc/disc_entropy                | 0.692    |
|    disc/disc_loss                   | 0.691    |
|    disc/disc_proportion_expert_pred | 0.92     |
|    disc/disc_proportion_expert_true | 0.5      |
|    disc/global_step                 | 1        |
|    disc/n_expert                    | 50       |
|    disc/n_generated                 | 50       |
-

round:   2%|▏         | 1/48 [00:06<04:53,  6.24s/it]

----------------------------------------------------
| raw/                               |             |
|    gen/rollout/ep_len_mean         | 50          |
|    gen/rollout/ep_rew_mean         | -38.6       |
|    gen/rollout/ep_rew_wrapped_mean | 33          |
|    gen/time/fps                    | 338         |
|    gen/time/iterations             | 1           |
|    gen/time/time_elapsed           | 6           |
|    gen/time/total_timesteps        | 4096        |
|    gen/train/approx_kl             | 0.008735466 |
|    gen/train/clip_fraction         | 0.0977      |
|    gen/train/clip_range            | 0.2         |
|    gen/train/entropy_loss          | -1.7        |
|    gen/train/explained_variance    | 0.251       |
|    gen/train/learning_rate         | 0.0001      |
|    gen/train/loss                  | 3.31        |
|    gen/train/n_updates             | 10          |
|    gen/train/policy_gradient_loss  | -0.018      |
|    gen/train/std                   | 0.564  

round:   4%|▍         | 2/48 [00:13<05:03,  6.60s/it]

...

round:  96%|█████████▌| 46/48 [04:59<00:13,  6.59s/it]

----------------------------------------------------
| raw/                               |             |
|    gen/rollout/ep_len_mean         | 50          |
|    gen/rollout/ep_rew_mean         | -27.3       |
|    gen/rollout/ep_rew_wrapped_mean | 8.54        |
|    gen/time/fps                    | 378         |
|    gen/time/iterations             | 1           |
|    gen/time/time_elapsed           | 5           |
|    gen/time/total_timesteps        | 96256       |
|    gen/train/approx_kl             | 0.010405403 |
|    gen/train/clip_fraction         | 0.108       |
|    gen/train/clip_range            | 0.2         |
|    gen/train/entropy_loss          | -0.935      |
|    gen/train/explained_variance    | 0.961       |
|    gen/train/learning_rate         | 0.0001      |
|    gen/train/loss                  | 1.42        |
|    gen/train/n_updates             | 460         |
|    gen/train/policy_gradient_loss  | -0.0191     |
|    gen/train/std                   | 0.386  

round:  98%|█████████▊| 47/48 [05:06<00:06,  6.47s/it]

----------------------------------------------------
| raw/                               |             |
|    gen/rollout/ep_len_mean         | 50          |
|    gen/rollout/ep_rew_mean         | -27.2       |
|    gen/rollout/ep_rew_wrapped_mean | 12.7        |
|    gen/time/fps                    | 350         |
|    gen/time/iterations             | 1           |
|    gen/time/time_elapsed           | 5           |
|    gen/time/total_timesteps        | 98304       |
|    gen/train/approx_kl             | 0.008500047 |
|    gen/train/clip_fraction         | 0.0521      |
|    gen/train/clip_range            | 0.2         |
|    gen/train/entropy_loss          | -0.927      |
|    gen/train/explained_variance    | 0.96        |
|    gen/train/learning_rate         | 0.0001      |
|    gen/train/loss                  | 2.24        |
|    gen/train/n_updates             | 470         |
|    gen/train/policy_gradient_loss  | -0.0114     |
|    gen/train/std                   | 0.383  

round: 100%|██████████| 48/48 [05:12<00:00,  6.51s/it]


In [64]:
env_4.seed(SEED)
learner_rewards_after_training, _ = evaluate_policy(
    learner_PPO_4, env_4, 20, return_episode_rewards=True
)

In [65]:
print(
    "Rewards before training:",
    np.mean(learner_rewards_before_training),
    "+/-",
    np.std(learner_rewards_before_training),
)
print(
    "learner rewards after training:",
    np.mean(learner_rewards_after_training),
    "+/-",
    np.std(learner_rewards_after_training),
)

Rewards before training: -5.9576569500000005 +/- 2.6330277433453007
learner rewards after training: -9.487759299999999 +/- 3.5814483312001184


# GAIL-PPO 10 trajectory - 1 traj BC

In [53]:
env_10, expert_10, rollouts_10 = make_env_expert_rollouts(10)
learner_PPO_10, gail_trainer_PPO_10 = make_PPO_learner_and_trainer(env_10, expert_10, rollouts_10)

Running with `allow_variable_horizon` set to True. Some algorithms are biased towards shorter or longer episodes, which may significantly confound results. Additionally, even unbiased algorithms can exploit the information leak from the termination condition, producing spuriously high performance. See https://imitation.readthedocs.io/en/latest/getting-started/variable-horizon.html for more information.


In [54]:
print(rollouts_10.count)

<built-in method count of list object at 0x7d312c323f40>


In [55]:
expert_reward, _ = evaluate_policy(
    expert_10, env_10, 20, return_episode_rewards=True
)
print(
    "expert rewards after training:",
    np.mean(expert_reward),
    "+/-",
    np.std(expert_reward),
)

expert rewards after training: -5.0036284 +/- 1.6795412761026864


In [56]:
env_10.seed(SEED)
learner_rewards_before_training, _ = evaluate_policy(
    learner_PPO_10, env_10, 20, return_episode_rewards=True
)

In [57]:
gail_trainer_PPO_10.train(100000) #进度条显示的是gen_train_timestep除以这里的input

round:   0%|          | 0/48 [00:00<?, ?it/s]

------------------------------------------
| raw/                        |          |
|    gen/rollout/ep_len_mean  | 50       |
|    gen/rollout/ep_rew_mean  | -37.7    |
|    gen/time/fps             | 370      |
|    gen/time/iterations      | 1        |
|    gen/time/time_elapsed    | 5        |
|    gen/time/total_timesteps | 2048     |
------------------------------------------
--------------------------------------------------
| raw/                                |          |
|    disc/disc_acc                    | 0.59     |
|    disc/disc_acc_expert             | 1        |
|    disc/disc_acc_gen                | 0.18     |
|    disc/disc_entropy                | 0.692    |
|    disc/disc_loss                   | 0.69     |
|    disc/disc_proportion_expert_pred | 0.91     |
|    disc/disc_proportion_expert_true | 0.5      |
|    disc/global_step                 | 1        |
|    disc/n_expert                    | 50       |
|    disc/n_generated                 | 50       |
-

round:   2%|▏         | 1/48 [00:06<05:05,  6.49s/it]

----------------------------------------------------
| raw/                               |             |
|    gen/rollout/ep_len_mean         | 50          |
|    gen/rollout/ep_rew_mean         | -38.6       |
|    gen/rollout/ep_rew_wrapped_mean | 33          |
|    gen/time/fps                    | 347         |
|    gen/time/iterations             | 1           |
|    gen/time/time_elapsed           | 5           |
|    gen/time/total_timesteps        | 4096        |
|    gen/train/approx_kl             | 0.008735466 |
|    gen/train/clip_fraction         | 0.0977      |
|    gen/train/clip_range            | 0.2         |
|    gen/train/entropy_loss          | -1.7        |
|    gen/train/explained_variance    | 0.251       |
|    gen/train/learning_rate         | 0.0001      |
|    gen/train/loss                  | 3.31        |
|    gen/train/n_updates             | 10          |
|    gen/train/policy_gradient_loss  | -0.018      |
|    gen/train/std                   | 0.564  

round:   4%|▍         | 2/48 [00:13<05:03,  6.60s/it]

...

round:  96%|█████████▌| 46/48 [05:00<00:13,  6.55s/it]

---------------------------------------------------
| raw/                               |            |
|    gen/rollout/ep_len_mean         | 50         |
|    gen/rollout/ep_rew_mean         | -26.5      |
|    gen/rollout/ep_rew_wrapped_mean | 5.33       |
|    gen/time/fps                    | 370        |
|    gen/time/iterations             | 1          |
|    gen/time/time_elapsed           | 5          |
|    gen/time/total_timesteps        | 96256      |
|    gen/train/approx_kl             | 0.01152863 |
|    gen/train/clip_fraction         | 0.106      |
|    gen/train/clip_range            | 0.2        |
|    gen/train/entropy_loss          | -0.844     |
|    gen/train/explained_variance    | 0.705      |
|    gen/train/learning_rate         | 0.0001     |
|    gen/train/loss                  | 0.0784     |
|    gen/train/n_updates             | 460        |
|    gen/train/policy_gradient_loss  | -0.0145    |
|    gen/train/std                   | 0.367      |
|    gen/tra

round:  98%|█████████▊| 47/48 [05:06<00:06,  6.49s/it]

-----------------------------------------------------
| raw/                               |              |
|    gen/rollout/ep_len_mean         | 50           |
|    gen/rollout/ep_rew_mean         | -26.6        |
|    gen/rollout/ep_rew_wrapped_mean | 5.39         |
|    gen/time/fps                    | 342          |
|    gen/time/iterations             | 1            |
|    gen/time/time_elapsed           | 5            |
|    gen/time/total_timesteps        | 98304        |
|    gen/train/approx_kl             | 0.0065751225 |
|    gen/train/clip_fraction         | 0.0555       |
|    gen/train/clip_range            | 0.2          |
|    gen/train/entropy_loss          | -0.817       |
|    gen/train/explained_variance    | 0.667        |
|    gen/train/learning_rate         | 0.0001       |
|    gen/train/loss                  | 0.238        |
|    gen/train/n_updates             | 470          |
|    gen/train/policy_gradient_loss  | -0.00958     |
|    gen/train/std          

round: 100%|██████████| 48/48 [05:13<00:00,  6.54s/it]


In [58]:
env_10.seed(SEED)
learner_rewards_after_training, _ = evaluate_policy(
    learner_PPO_10, env_10, 20, return_episode_rewards=True
)

In [59]:
print(
    "Rewards before training:",
    np.mean(learner_rewards_before_training),
    "+/-",
    np.std(learner_rewards_before_training),
)
print(
    "learner rewards after training:",
    np.mean(learner_rewards_after_training),
    "+/-",
    np.std(learner_rewards_after_training),
)

Rewards before training: -5.9576569500000005 +/- 2.6330277433453007
learner rewards after training: -11.432267849999999 +/- 4.270106992960519


# give trainer 10 traj BC network

In [28]:
import torch
def make_PPO_learner_and_trainer_10(env, expert, rollouts):
    global SEED
# 可以调policy network的架构
# Custom actor (pi) and value function (vf) networks
# of two layers of size 32 each with Relu activation function
#policy_kwargs = dict(activation_fn=th.nn.ReLU,
#                     net_arch=[dict(pi=[32, 32], vf=[32, 32])])
    learner_PPO = PPO(
        env=env,
        policy=MlpPolicy,
        batch_size=128,
        #clip_range=0.3,
        #ent_coef= 0.0007566389899529574,
        learning_rate=1e-4, #1.943992487657563e-5
        gae_lambda=0.8,
        max_grad_norm=0.9,
        gamma=0.995,
        n_epochs=10,
        seed=SEED,
    )
    load_from_saved_policy = torch.load("BC_MlpPolicy_Reacher_10.pth.tar")
    learner_PPO.policy.load_state_dict(load_from_saved_policy.state_dict(), strict=False)

    reward_net = BasicRewardNet(
        observation_space=env.observation_space,
        action_space=env.action_space,
        normalize_input_layer=RunningNorm,
    )
    gail_trainer = GAIL(
        demonstrations = rollouts, # expert trajectories
        demo_batch_size = 50,
        gen_replay_buffer_capacity = 16384,
        n_disc_updates_per_round = 8, #
        venv = env,
        gen_algo = learner_PPO,
        reward_net=reward_net,
        allow_variable_horizon=True,
        # gen_train_timesteps = 1000
        init_tensorboard=True,
        init_tensorboard_graph=True,

    )
    return learner_PPO, gail_trainer

# GAIL-PPO 1 trajectory - 10 traj BC

In [29]:
env_1, expert_1, rollouts_1 = make_env_expert_rollouts(1)

In [30]:
learner_PPO_1, gail_trainer_PPO_1 = make_PPO_learner_and_trainer_10(env_1, expert_1, rollouts_1)

Running with `allow_variable_horizon` set to True. Some algorithms are biased towards shorter or longer episodes, which may significantly confound results. Additionally, even unbiased algorithms can exploit the information leak from the termination condition, producing spuriously high performance. See https://imitation.readthedocs.io/en/latest/getting-started/variable-horizon.html for more information.


In [31]:
expert_reward, _ = evaluate_policy(
    expert_1, env_1, 20, return_episode_rewards=True
)
print(
    "expert rewards after training:",
    np.mean(expert_reward),
    "+/-",
    np.std(expert_reward),
)

expert rewards after training: -5.0036284 +/- 1.6795412761026864


In [32]:
env_1.seed(SEED)
learner_rewards_before_training, _ = evaluate_policy(
    learner_PPO_1, env_1, 20, return_episode_rewards=True
)

In [33]:
gail_trainer_PPO_1.train(100000)
# An upper bound on the number of transitions to sample from the environment during training.

round:   0%|          | 0/48 [00:00<?, ?it/s]

------------------------------------------
| raw/                        |          |
|    gen/rollout/ep_len_mean  | 50       |
|    gen/rollout/ep_rew_mean  | -33.9    |
|    gen/time/fps             | 344      |
|    gen/time/iterations      | 1        |
|    gen/time/time_elapsed    | 5        |
|    gen/time/total_timesteps | 2048     |
------------------------------------------
--------------------------------------------------
| raw/                                |          |
|    disc/disc_acc                    | 0.57     |
|    disc/disc_acc_expert             | 1        |
|    disc/disc_acc_gen                | 0.14     |
|    disc/disc_entropy                | 0.692    |
|    disc/disc_loss                   | 0.693    |
|    disc/disc_proportion_expert_pred | 0.93     |
|    disc/disc_proportion_expert_true | 0.5      |
|    disc/global_step                 | 1        |
|    disc/n_expert                    | 50       |
|    disc/n_generated                 | 50       |
-

round:   2%|▏         | 1/48 [00:06<05:17,  6.76s/it]

-----------------------------------------------------
| raw/                               |              |
|    gen/rollout/ep_len_mean         | 50           |
|    gen/rollout/ep_rew_mean         | -34.6        |
|    gen/rollout/ep_rew_wrapped_mean | 34           |
|    gen/time/fps                    | 370          |
|    gen/time/iterations             | 1            |
|    gen/time/time_elapsed           | 5            |
|    gen/time/total_timesteps        | 4096         |
|    gen/train/approx_kl             | 0.0097206235 |
|    gen/train/clip_fraction         | 0.082        |
|    gen/train/clip_range            | 0.2          |
|    gen/train/entropy_loss          | -1.52        |
|    gen/train/explained_variance    | -0.102       |
|    gen/train/learning_rate         | 0.0001       |
|    gen/train/loss                  | 3.32         |
|    gen/train/n_updates             | 10           |
|    gen/train/policy_gradient_loss  | -0.0121      |
|    gen/train/std          

round:   4%|▍         | 2/48 [00:13<05:00,  6.54s/it]

...

round:  96%|█████████▌| 46/48 [05:05<00:13,  6.61s/it]

----------------------------------------------------
| raw/                               |             |
|    gen/rollout/ep_len_mean         | 50          |
|    gen/rollout/ep_rew_mean         | -28.5       |
|    gen/rollout/ep_rew_wrapped_mean | 2.22        |
|    gen/time/fps                    | 379         |
|    gen/time/iterations             | 1           |
|    gen/time/time_elapsed           | 5           |
|    gen/time/total_timesteps        | 96256       |
|    gen/train/approx_kl             | 0.015066653 |
|    gen/train/clip_fraction         | 0.195       |
|    gen/train/clip_range            | 0.2         |
|    gen/train/entropy_loss          | -0.909      |
|    gen/train/explained_variance    | 0.991       |
|    gen/train/learning_rate         | 0.0001      |
|    gen/train/loss                  | 0.358       |
|    gen/train/n_updates             | 460         |
|    gen/train/policy_gradient_loss  | -0.0268     |
|    gen/train/std                   | 0.381  

round:  98%|█████████▊| 47/48 [05:11<00:06,  6.49s/it]

----------------------------------------------------
| raw/                               |             |
|    gen/rollout/ep_len_mean         | 50          |
|    gen/rollout/ep_rew_mean         | -28.9       |
|    gen/rollout/ep_rew_wrapped_mean | 1.67        |
|    gen/time/fps                    | 344         |
|    gen/time/iterations             | 1           |
|    gen/time/time_elapsed           | 5           |
|    gen/time/total_timesteps        | 98304       |
|    gen/train/approx_kl             | 0.015501706 |
|    gen/train/clip_fraction         | 0.156       |
|    gen/train/clip_range            | 0.2         |
|    gen/train/entropy_loss          | -0.904      |
|    gen/train/explained_variance    | 0.988       |
|    gen/train/learning_rate         | 0.0001      |
|    gen/train/loss                  | 0.348       |
|    gen/train/n_updates             | 470         |
|    gen/train/policy_gradient_loss  | -0.0252     |
|    gen/train/std                   | 0.38   

round: 100%|██████████| 48/48 [05:18<00:00,  6.64s/it]


In [34]:
env_1.seed(SEED)
learner_rewards_after_training, _ = evaluate_policy(
    learner_PPO_1, env_1, 20, return_episode_rewards=True
)

In [35]:
print(
    "Rewards before training:",
    np.mean(learner_rewards_before_training),
    "+/-",
    np.std(learner_rewards_before_training),
)
print(
    "Rewards after training:",
    np.mean(learner_rewards_after_training),
    "+/-",
    np.std(learner_rewards_after_training),
)

Rewards before training: -5.841108450000001 +/- 2.22987443417024
Rewards after training: -12.7155241 +/- 3.2591501024244636


# GAIL-PPO 4 trajectory - 10 traj BC

In [66]:
env_4, expert_4, rollouts_4 = make_env_expert_rollouts(4)
learner_PPO_4, gail_trainer_PPO_4 = make_PPO_learner_and_trainer_10(env_4, expert_4, rollouts_4)

Running with `allow_variable_horizon` set to True. Some algorithms are biased towards shorter or longer episodes, which may significantly confound results. Additionally, even unbiased algorithms can exploit the information leak from the termination condition, producing spuriously high performance. See https://imitation.readthedocs.io/en/latest/getting-started/variable-horizon.html for more information.


In [67]:
expert_reward, _ = evaluate_policy(
    expert_4, env_4, 20, return_episode_rewards=True
)
print(
    "expert rewards after training:",
    np.mean(expert_reward),
    "+/-",
    np.std(expert_reward),
)

expert rewards after training: -5.0036284 +/- 1.6795412761026864


In [68]:
env_4.seed(SEED)
learner_rewards_before_training, _ = evaluate_policy(
    learner_PPO_4, env_4, 20, return_episode_rewards=True
)

In [69]:
gail_trainer_PPO_4.train(100000) # 1000000

round:   0%|          | 0/48 [00:00<?, ?it/s]

------------------------------------------
| raw/                        |          |
|    gen/rollout/ep_len_mean  | 50       |
|    gen/rollout/ep_rew_mean  | -33.9    |
|    gen/time/fps             | 357      |
|    gen/time/iterations      | 1        |
|    gen/time/time_elapsed    | 5        |
|    gen/time/total_timesteps | 2048     |
------------------------------------------
--------------------------------------------------
| raw/                                |          |
|    disc/disc_acc                    | 0.56     |
|    disc/disc_acc_expert             | 1        |
|    disc/disc_acc_gen                | 0.12     |
|    disc/disc_entropy                | 0.692    |
|    disc/disc_loss                   | 0.694    |
|    disc/disc_proportion_expert_pred | 0.94     |
|    disc/disc_proportion_expert_true | 0.5      |
|    disc/global_step                 | 1        |
|    disc/n_expert                    | 50       |
|    disc/n_generated                 | 50       |
-

round:   2%|▏         | 1/48 [00:06<05:05,  6.51s/it]

-----------------------------------------------------
| raw/                               |              |
|    gen/rollout/ep_len_mean         | 50           |
|    gen/rollout/ep_rew_mean         | -34.6        |
|    gen/rollout/ep_rew_wrapped_mean | 34           |
|    gen/time/fps                    | 375          |
|    gen/time/iterations             | 1            |
|    gen/time/time_elapsed           | 5            |
|    gen/time/total_timesteps        | 4096         |
|    gen/train/approx_kl             | 0.0097206235 |
|    gen/train/clip_fraction         | 0.082        |
|    gen/train/clip_range            | 0.2          |
|    gen/train/entropy_loss          | -1.52        |
|    gen/train/explained_variance    | -0.102       |
|    gen/train/learning_rate         | 0.0001       |
|    gen/train/loss                  | 3.32         |
|    gen/train/n_updates             | 10           |
|    gen/train/policy_gradient_loss  | -0.0121      |
|    gen/train/std          

round:   4%|▍         | 2/48 [00:12<04:57,  6.47s/it]

...

round:  96%|█████████▌| 46/48 [04:57<00:12,  6.49s/it]

-----------------------------------------------------
| raw/                               |              |
|    gen/rollout/ep_len_mean         | 50           |
|    gen/rollout/ep_rew_mean         | -24.4        |
|    gen/rollout/ep_rew_wrapped_mean | 5.46         |
|    gen/time/fps                    | 363          |
|    gen/time/iterations             | 1            |
|    gen/time/time_elapsed           | 5            |
|    gen/time/total_timesteps        | 96256        |
|    gen/train/approx_kl             | 0.0041952105 |
|    gen/train/clip_fraction         | 0.0342       |
|    gen/train/clip_range            | 0.2          |
|    gen/train/entropy_loss          | -0.702       |
|    gen/train/explained_variance    | 0.995        |
|    gen/train/learning_rate         | 0.0001       |
|    gen/train/loss                  | 0.43         |
|    gen/train/n_updates             | 460          |
|    gen/train/policy_gradient_loss  | -0.00653     |
|    gen/train/std          

round:  98%|█████████▊| 47/48 [05:04<00:06,  6.56s/it]

----------------------------------------------------
| raw/                               |             |
|    gen/rollout/ep_len_mean         | 50          |
|    gen/rollout/ep_rew_mean         | -24         |
|    gen/rollout/ep_rew_wrapped_mean | 5.51        |
|    gen/time/fps                    | 379         |
|    gen/time/iterations             | 1           |
|    gen/time/time_elapsed           | 5           |
|    gen/time/total_timesteps        | 98304       |
|    gen/train/approx_kl             | 0.006319595 |
|    gen/train/clip_fraction         | 0.0514      |
|    gen/train/clip_range            | 0.2         |
|    gen/train/entropy_loss          | -0.683      |
|    gen/train/explained_variance    | 0.995       |
|    gen/train/learning_rate         | 0.0001      |
|    gen/train/loss                  | 0.308       |
|    gen/train/n_updates             | 470         |
|    gen/train/policy_gradient_loss  | -0.00681    |
|    gen/train/std                   | 0.338  

round: 100%|██████████| 48/48 [05:10<00:00,  6.47s/it]


In [70]:
env_4.seed(SEED)
learner_rewards_after_training, _ = evaluate_policy(
    learner_PPO_4, env_4, 20, return_episode_rewards=True
)

In [71]:
print(
    "Rewards before training:",
    np.mean(learner_rewards_before_training),
    "+/-",
    np.std(learner_rewards_before_training),
)
print(
    "learner rewards after training:",
    np.mean(learner_rewards_after_training),
    "+/-",
    np.std(learner_rewards_after_training),
)

Rewards before training: -5.841108450000001 +/- 2.22987443417024
learner rewards after training: -9.6991937 +/- 2.128143902265401


# GAIL-PPO 10 trajectory - 10 traj BC

In [42]:
env_10, expert_10, rollouts_10 = make_env_expert_rollouts(10)
learner_PPO_10, gail_trainer_PPO_10 = make_PPO_learner_and_trainer_10(env_10, expert_10, rollouts_10)

Running with `allow_variable_horizon` set to True. Some algorithms are biased towards shorter or longer episodes, which may significantly confound results. Additionally, even unbiased algorithms can exploit the information leak from the termination condition, producing spuriously high performance. See https://imitation.readthedocs.io/en/latest/getting-started/variable-horizon.html for more information.


In [43]:
print(rollouts_10.count)

<built-in method count of list object at 0x7d311c5d7ec0>


In [44]:
expert_reward, _ = evaluate_policy(
    expert_10, env_10, 20, return_episode_rewards=True
)
print(
    "expert rewards after training:",
    np.mean(expert_reward),
    "+/-",
    np.std(expert_reward),
)

expert rewards after training: -5.0036284 +/- 1.6795412761026864


In [45]:
env_10.seed(SEED)
learner_rewards_before_training, _ = evaluate_policy(
    learner_PPO_10, env_10, 20, return_episode_rewards=True
)

In [46]:
gail_trainer_PPO_10.train(100000) #进度条显示的是gen_train_timestep除以这里的input

round:   0%|          | 0/48 [00:00<?, ?it/s]

------------------------------------------
| raw/                        |          |
|    gen/rollout/ep_len_mean  | 50       |
|    gen/rollout/ep_rew_mean  | -33.9    |
|    gen/time/fps             | 346      |
|    gen/time/iterations      | 1        |
|    gen/time/time_elapsed    | 5        |
|    gen/time/total_timesteps | 2048     |
------------------------------------------
--------------------------------------------------
| raw/                                |          |
|    disc/disc_acc                    | 0.57     |
|    disc/disc_acc_expert             | 1        |
|    disc/disc_acc_gen                | 0.14     |
|    disc/disc_entropy                | 0.692    |
|    disc/disc_loss                   | 0.693    |
|    disc/disc_proportion_expert_pred | 0.93     |
|    disc/disc_proportion_expert_true | 0.5      |
|    disc/global_step                 | 1        |
|    disc/n_expert                    | 50       |
|    disc/n_generated                 | 50       |
-

round:   2%|▏         | 1/48 [00:06<05:14,  6.69s/it]

-----------------------------------------------------
| raw/                               |              |
|    gen/rollout/ep_len_mean         | 50           |
|    gen/rollout/ep_rew_mean         | -34.6        |
|    gen/rollout/ep_rew_wrapped_mean | 34           |
|    gen/time/fps                    | 372          |
|    gen/time/iterations             | 1            |
|    gen/time/time_elapsed           | 5            |
|    gen/time/total_timesteps        | 4096         |
|    gen/train/approx_kl             | 0.0097206235 |
|    gen/train/clip_fraction         | 0.082        |
|    gen/train/clip_range            | 0.2          |
|    gen/train/entropy_loss          | -1.52        |
|    gen/train/explained_variance    | -0.102       |
|    gen/train/learning_rate         | 0.0001       |
|    gen/train/loss                  | 3.32         |
|    gen/train/n_updates             | 10           |
|    gen/train/policy_gradient_loss  | -0.0121      |
|    gen/train/std          

round:   4%|▍         | 2/48 [00:13<05:02,  6.57s/it]

...

round:  96%|█████████▌| 46/48 [05:01<00:13,  6.77s/it]

----------------------------------------------------
| raw/                               |             |
|    gen/rollout/ep_len_mean         | 50          |
|    gen/rollout/ep_rew_mean         | -21.9       |
|    gen/rollout/ep_rew_wrapped_mean | 5.66        |
|    gen/time/fps                    | 286         |
|    gen/time/iterations             | 1           |
|    gen/time/time_elapsed           | 7           |
|    gen/time/total_timesteps        | 96256       |
|    gen/train/approx_kl             | 0.007452622 |
|    gen/train/clip_fraction         | 0.0818      |
|    gen/train/clip_range            | 0.2         |
|    gen/train/entropy_loss          | -0.544      |
|    gen/train/explained_variance    | 0.735       |
|    gen/train/learning_rate         | 0.0001      |
|    gen/train/loss                  | 0.253       |
|    gen/train/n_updates             | 460         |
|    gen/train/policy_gradient_loss  | -0.0125     |
|    gen/train/std                   | 0.316  

round:  98%|█████████▊| 47/48 [05:09<00:07,  7.25s/it]

---------------------------------------------------
| raw/                               |            |
|    gen/rollout/ep_len_mean         | 50         |
|    gen/rollout/ep_rew_mean         | -21.3      |
|    gen/rollout/ep_rew_wrapped_mean | 7.39       |
|    gen/time/fps                    | 261        |
|    gen/time/iterations             | 1          |
|    gen/time/time_elapsed           | 7          |
|    gen/time/total_timesteps        | 98304      |
|    gen/train/approx_kl             | 0.00844845 |
|    gen/train/clip_fraction         | 0.0928     |
|    gen/train/clip_range            | 0.2        |
|    gen/train/entropy_loss          | -0.52      |
|    gen/train/explained_variance    | 0.598      |
|    gen/train/learning_rate         | 0.0001     |
|    gen/train/loss                  | 0.42       |
|    gen/train/n_updates             | 470        |
|    gen/train/policy_gradient_loss  | -0.0124    |
|    gen/train/std                   | 0.312      |
|    gen/tra

round: 100%|██████████| 48/48 [05:18<00:00,  6.63s/it]


In [47]:
env_10.seed(SEED)
learner_rewards_after_training, _ = evaluate_policy(
    learner_PPO_10, env_10, 20, return_episode_rewards=True
)

In [51]:
print(
    "learner rewards before training:",
    np.mean(learner_rewards_before_training),
    "+/-",
    np.std(learner_rewards_before_training),
)

learner rewards before training: -5.841108450000001 +/- 2.22987443417024


In [52]:
print(
    "learner rewards after training:",
    np.mean(learner_rewards_after_training),
    "+/-",
    np.std(learner_rewards_after_training),
)

learner rewards after training: -8.216863 +/- 2.5059840901378845
